In [1]:
import cv2, os, joblib
import numpy as np
from pathlib import Path
from keras.models import load_model

Using TensorFlow backend.


In [75]:
def Learning():
    import cv2
    from pathlib import Path
    from keras.models import Model, Sequential
    from keras.preprocessing import image
    from keras.applications import vgg16
    from keras.preprocessing.image import ImageDataGenerator
    from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Dropout, Activation, Flatten
    from keras.optimizers import Adam


    img_rows, img_cols = 224, 224 
    base_model =vgg16.VGG16(weights = 'imagenet', 
                     include_top = False, 
                     input_shape = (img_rows, img_cols, 3))


    # Here we freeze the last 4 layers 
    # Layers are set to trainable as True by default
    for layer in base_model.layers:
        layer.trainable = False
    #model.summary()


    def add(BottomModel, Class):
        x = BottomModel.output
        x = GlobalAveragePooling2D()(x)
        x = Flatten()(x)
        x = Dense(1024,activation='relu')(x)
        x = Dense(1024,activation='relu')(x)
        x = Dense(512,activation='relu')(x)
        x = Dense(Class,activation='softmax')(x)
        return x


    Class = 2
    top = add(base_model, Class)
    model = Model(inputs = base_model.input, outputs = top)


    #model.summary()


    dir = Path('C://Users//kevin//Documents//Project//Task_4//cell_images')
    #Image Data Generator
    train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            dir / 'training_set',
            target_size=(img_rows, img_cols),
            batch_size=16,
            class_mode='categorical')

    
    validation_generator = validation_datagen.flow_from_directory(
        dir / 'testing_set',
        target_size=(img_rows, img_cols),
        batch_size=16,
        class_mode='categorical')

    # We use a very small learning rate 
    optimizer = Adam(lr=0.001)
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer,
                  metrics = ['accuracy'])

    #validation_steps < validation_dataset_size / batch_size
    history = model.fit_generator(
    train_generator,
    epochs = 5,
    validation_data = validation_generator,
    )

    model.save('TL_Model.h5')

In [76]:
def Test():    
    import os, random
    from PIL import Image, ImageFont, ImageDraw
    from pathlib import Path
    from keras.models import load_model
    from keras.preprocessing import image

    path = Path("C://Users//kevin//Documents//Project//Task_4//cell_images//prediction_set")
    fold = random.choice(os.listdir(path)) 
    im = Image.open(path / fold)
    im = im.resize((224,224), resample = Image.NEAREST)
    npimage = image.img_to_array(im)
    npimage = np.expand_dims(npimage, axis=0)

    model = load_model('TL_Model.h5')
    
    result = model.predict(npimage)
    if result[0][0] == 1.0:
        d = ImageDraw.Draw(im)
        d.text((10,10), "Parasitized", fill=(255,255,0))
        im.save('pil_twext.png')
        newim = Image.open('pil_twext.png')
        newim.show()
    else:
        d = ImageDraw.Draw(im)
        d.text((10,10), "Healthy", fill=(255,255,0))
        im.save('pil_twwext.png')
        parim = Image.open('pil_twwext.png')
        parim.show()

In [77]:
#print('Yay, the file exists!') if k.A.exists() and k.B.exists() else print("Oops, file doesn't exist!")

In [78]:
Choice = True
while Choice:
    print("""
    1. Train Model
    2. Test Model
    3. Exit
    """)
    Choice = input("What would you like to do? ")

    if Choice == "1":
        Learning()
        print("\nModel Trained")
    elif Choice == "2":
        Test()
        print("\nTest Successful")
    elif Choice == "3":
      print("\n Goodbye") 
      Choice = None
    else:
       print("\n Not Valid Choice Try again")


    1. Train Model
    2. Test Model
    3. Exit
    
What would you like to do? 1
Found 1200 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/5
75/75 [==============================] - 335s 4s/step - loss: 0.7628 - accuracy: 0.5242 - val_loss: 0.6659 - val_accuracy: 0.5275
Epoch 2/5
75/75 [==============================] - 534s 7s/step - loss: 0.6410 - accuracy: 0.6300 - val_loss: 0.4391 - val_accuracy: 0.8400
Epoch 3/5
75/75 [==============================] - 614s 8s/step - loss: 0.5280 - accuracy: 0.7317 - val_loss: 0.3507 - val_accuracy: 0.8775
Epoch 4/5
75/75 [==============================] - 639s 9s/step - loss: 0.5058 - accuracy: 0.7533 - val_loss: 0.4967 - val_accuracy: 0.7225
Epoch 5/5
75/75 [==============================] - 447s 6s/step - loss: 0.4396 - accuracy: 0.8042 - val_loss: 0.3418 - val_accuracy: 0.8775

Model Trained

    1. Train Model
    2. Test Model
    3. Exit
    
What would you like to do? 2

Test Successful

    1. Train Mod